In [1]:
!pip install pyspark

from pyspark.sql import SparkSession


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=6790ab854e6a9ad80d7185ed3e4c5f1e02150ed070448c005ecf15128ad0c157
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate() #Seteaza numarul de core-uri de utilizat (* = toate)

csv_path = "/content/Students/StudentsPerformance/" #Locatia datelor noastre

df = spark.read.csv(csv_path, header=True, inferSchema=True) #Incarcarea datelor din CSV intr-un dataframe Spark

In [3]:
table_name = "my_table" #definirea unei variabile table_name
df.createOrReplaceTempView(table_name) #crearea unui view temporar
query = "SELECT * FROM my_table" #definirea unei interogari SQL
result = spark.sql(query) #rularea interogarii
result.show() #afisarea rezultatelor

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

In [4]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



In [5]:
df.describe().show()


+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|parental level of education|       lunch|test preparation course|        math score|     reading score|    writing score|
+-------+------+--------------+---------------------------+------------+-----------------------+------------------+------------------+-----------------+
|  count|  1000|          1000|                       1000|        1000|                   1000|              1000|              1000|             1000|
|   mean|  null|          null|                       null|        null|                   null|            66.089|            69.169|           68.054|
| stddev|  null|          null|                       null|        null|                   null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|         associate's degree|free/reduced|          

In [6]:
df = df.withColumnRenamed('parental level of education', 'parentEducation')
df = df.withColumnRenamed('test preparation course', 'preparationCourse')
df = df.withColumnRenamed('math score', 'mathScore')
df = df.withColumnRenamed('reading score', 'readingScore')
df = df.withColumnRenamed('writing score', 'writingScore')
df.describe().show()
df.write.format('csv').mode('overwrite').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted')


+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|summary|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|         mathScore|      readingScore|     writingScore|
+-------+------+--------------+------------------+------------+-----------------+------------------+------------------+-----------------+
|  count|  1000|          1000|              1000|        1000|             1000|              1000|              1000|             1000|
|   mean|  null|          null|              null|        null|             null|            66.089|            69.169|           68.054|
| stddev|  null|          null|              null|        null|             null|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|female|       group A|associate's degree|free/reduced|        completed|                 0|                17|               10|
|    max|  male|       group E|  s

In [7]:
csv_path = "/content/Students/StudentsPerformanceFormatted" #salvam locatia csv-ului formatat intr-o noua variabila
df = spark.read.csv(csv_path, header=True, inferSchema=True) #incarcam noile date
from pyspark.sql.functions import col, round, format_number, when #importam lucrul cu coloane

df = df.withColumn('finalScore', (col('mathScore') + col('readingScore') + col('writingScore'))/3) #mai adaugam o coloana care determina media generala
df = df.withColumn('finalScore', format_number(col('finalScore'), 2)) #afisam doar 2 decimale in aceasta noua coloana
df = df.withColumn('admited', when(col('finalScore') > 50 , True).otherwise(False))#mai adaugam o coloana care va afisa true pentru cazul in care studentul a fost admis(finalScore > 50) sau false pentru cazul in care nu a fost admis(finalScore < 50)
df.show()#afisam rezultatele
#Salvam dataframe-ul modificat 
df.write.mode('overwrite').format('csv').option('header', 'true').save('/content/Students/StudentsPerformanceFormatted2')

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 

In [8]:
df.printSchema() #afisarea schemei datelor

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parentEducation: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- preparationCourse: string (nullable = true)
 |-- mathScore: integer (nullable = true)
 |-- readingScore: integer (nullable = true)
 |-- writingScore: integer (nullable = true)
 |-- finalScore: string (nullable = true)
 |-- admited: boolean (nullable = false)



In [9]:

 
#salvam locatia csv-ului formatat intr-o noua variabila
csv_path = "/content/Students/StudentsPerformanceFormatted2"

#incarcam noile date
df = spark.read.csv(csv_path, header=True, inferSchema=True) 

df.show()

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group C|      some college|    standard|        completed|       69|          90|          88|     82.33|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group A|associate's degree|free/reduced|             none|       47|          57|          44|     49.33|  false|
|  male|       group C|      some college|    standard|             none|       76|          78| 

SPARK SQL

1.Sa se afiseze toti studenții care nu au obținut o nota de trecere la matematica ordonand de la cea mai mica nota

In [10]:
df.createOrReplaceTempView("students")

result = spark.sql("SELECT * FROM students WHERE mathScore < 50 ORDER BY mathScore ASC")
result.show()


+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group C|  some high school|free/reduced|             none|        0|          17|          10|       9.0|  false|
|female|       group B|       high school|free/reduced|             none|        8|          24|          23|     18.33|  false|
|female|       group B|  some high school|free/reduced|             none|       18|          32|          28|      26.0|  false|
|female|       group B|      some college|    standard|             none|       19|          38|          32|     29.67|  false|
|female|       group C|      some college|free/reduced|             none|       22|          39| 

2.Sa se afiseze studentii care nu au fost admiși și nivelul de pregătire al părinților

In [11]:
result = spark.sql("SELECT admited, parentEducation FROM students WHERE admited == 'false' ORDER BY parentEducation")

result.show()


+-------+------------------+
|admited|   parentEducation|
+-------+------------------+
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
|  false|associate's degree|
+-------+------------------+
only showing top 20 rows



3. Sa se afiseze numarul de studenti care au fost admisi

In [12]:
result = spark.sql("SELECT COUNT(*) AS TotalAdmisi FROM students WHERE admited == 'true'")

result.show()


+-----------+
|TotalAdmisi|
+-----------+
|        880|
+-----------+



4. Sa se afiseze numarul total de studenti respinsi

In [13]:
result = spark.sql("SELECT COUNT(*) AS TotalRespinsi FROM students WHERE admited == 'false'")

result.show()


+-------------+
|TotalRespinsi|
+-------------+
|          120|
+-------------+



5. Sa se afișeze studenții care nu au fost admiși dar, totuși, au urmat cursul de pregătire 

In [14]:
result = spark.sql("SELECT * FROM students WHERE admited == 'false' AND preparationCourse == 'completed' ")

result.show()


+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|  male|       group C|   master's degree|free/reduced|        completed|       46|          42|          46|     44.67|  false|
|  male|       group A|      some college|free/reduced|        completed|       50|          47|          54|     50.33|  false|
|  male|       group C|associate's degree|free/reduced|        completed|       43|          45|          50|      46.0|  false|
|  male|       group B|       high school|    standard|        completed|       60|          44|          47|     50.33|  false|
|  male|       group C|  some high school|free/reduced|        completed|       53|          37| 

6. Sa se afiseze nota medie la matematica pe fiecare gen:

In [15]:
result = spark.sql("SELECT gender, FORMAT_NUMBER(AVG(mathScore), 2) AS NotaMedieLaMatematica FROM students GROUP BY gender")
result.show()



+------+---------------------+
|gender|NotaMedieLaMatematica|
+------+---------------------+
|female|                63.63|
|  male|                68.73|
+------+---------------------+



7. Sa se calculeze nota medie la citit pe fiecare rasa/etnicitate

In [16]:
result = spark.sql("SELECT `race/ethnicity`, FORMAT_NUMBER(AVG(readingScore), 2) AS NotaMedieLaCitit FROM students GROUP BY `race/ethnicity` ")
result.show()

+--------------+----------------+
|race/ethnicity|NotaMedieLaCitit|
+--------------+----------------+
|       group B|           67.35|
|       group C|           69.10|
|       group D|           70.03|
|       group A|           64.67|
|       group E|           73.03|
+--------------+----------------+



8. Sa se gaseasca numeral total de students pentru fiecare nivel de educatie al parintilor

In [17]:
result = spark.sql("SELECT parentEducation, COUNT(*) AS student_count FROM students GROUP BY parentEducation")
result.show()

+------------------+-------------+
|   parentEducation|student_count|
+------------------+-------------+
|  some high school|          179|
|associate's degree|          222|
|       high school|          196|
| bachelor's degree|          118|
|   master's degree|           59|
|      some college|          226|
+------------------+-------------+



9. Sa se afiseze numeral de studenti admisi si procentajul pentru fiecare tip de prânz

In [18]:
result = spark.sql("SELECT lunch,  COUNT(*) AS totalStudents, SUM(CASE WHEN admited = 'Yes' THEN 1 ELSE 0 END) AS studentiAdmisi, FORMAT_NUMBER((SUM(CASE WHEN admited = 'Yes' THEN 1 ELSE 0 END) / COUNT(*)) * 100, 2) AS procentajAdmisi FROM students GROUP BY lunch")
result.show()

+------------+-------------+--------------+---------------+
|       lunch|totalStudents|studentiAdmisi|procentajAdmisi|
+------------+-------------+--------------+---------------+
|free/reduced|          355|           283|          79.72|
|    standard|          645|           597|          92.56|
+------------+-------------+--------------+---------------+



10. Sa se afiseze nota medie la scris pe gen pentru studentii care au completat cursul de pregatire

In [19]:
result = spark.sql("SELECT gender, FORMAT_NUMBER(AVG(writingScore), 2) AS notaMedieLaScris FROM students WHERE preparationCourse = 'completed' GROUP BY gender")
result.show()

+------+----------------+
|gender|notaMedieLaScris|
+------+----------------+
|female|           78.79|
|  male|           69.79|
+------+----------------+



Fltrari de data folosind Dataframes

1. Afisati toti studentii care nu au urmat cursul de pregatire, dar,totusi, au fost admisi.

In [20]:
df.filter((df.preparationCourse == 'none') & (df.admited == 'true')).show()

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|female|       group B| bachelor's degree|    standard|             none|       72|          72|          74|     72.67|   true|
|female|       group B|   master's degree|    standard|             none|       90|          95|          93|     92.67|   true|
|  male|       group C|      some college|    standard|             none|       76|          78|          75|     76.33|   true|
|female|       group B|associate's degree|    standard|             none|       71|          83|          78|     77.33|   true|
|  male|       group C|associate's degree|    standard|             none|       58|          54| 

2. Afisati toti studentii care nu au reusit sa obtina o nota de trecere la niciuna dintre materii

In [21]:
df.filter((df.mathScore < 50) & (df.writingScore < 50 ) & (df.readingScore < 50 )).show()

+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|gender|race/ethnicity|   parentEducation|       lunch|preparationCourse|mathScore|readingScore|writingScore|finalScore|admited|
+------+--------------+------------------+------------+-----------------+---------+------------+------------+----------+-------+
|  male|       group B|      some college|free/reduced|             none|       40|          43|          39|     40.67|  false|
|female|       group B|  some high school|free/reduced|             none|       18|          32|          28|      26.0|  false|
|  male|       group C|   master's degree|free/reduced|        completed|       46|          42|          46|     44.67|  false|
|  male|       group D|      some college|    standard|             none|       40|          42|          38|      40.0|  false|
|female|       group C|       high school|free/reduced|             none|       33|          41| 

Machine Learning

1. Vom aplica reglesia liniara pentru a estima nota unui student la citit bazandu-ne pe nivelul de educatie al parintilor, nota la scris si in functie de testul de pregatire.

In [22]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

# Incarcam datele formatate
data = spark.read.csv('/content/Students/StudentsPerformanceFormatted2/', header=True, inferSchema=True)

# Partea de preprocesare - Convertirea coloanelor categorice in numerice

# Definim coloanele categorice pe care le vom folosi
categorical_cols = ['parentEducation', 'preparationCourse']

# Cream o lista pentru stocare
indexers = []
# Aplicam StringIndexer-ul pentru fiecare coloana categorica
for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col + 'Index')
    indexers.append(indexer)

#Aplicam toti index-erii de mai sus intr-un mod secvential folosind un pipeline

pipeline = Pipeline(stages=indexers)
data = pipeline.fit(data).transform(data)

data.printSchema()

# Selectarea coloanelor si a coloanei tinta
feature_cols = ['parentEducationIndex', 'writingScore', 'preparationCourseIndex']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
data = assembler.transform(data).select('features', 'readingScore')

train_data, test_data = data.randomSplit([0.8, 0.2], seed=123)

lr = LinearRegression(featuresCol='features', labelCol='readingScore')
lr_model = lr.fit(train_data)

predictions = lr_model.transform(test_data)
predictions.select('features', 'readingScore', 'prediction').show()

# Putem, de asemenea, calcula metricele de evaluare RMSE(root mean squared error) sau R-squared

evaluator = RegressionEvaluator(labelCol='readingScore', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(predictions)
print('Root Mean Squared Error (RMSE):', rmse)

r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
print('R-squared:', r2)



root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parentEducation: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- preparationCourse: string (nullable = true)
 |-- mathScore: integer (nullable = true)
 |-- readingScore: integer (nullable = true)
 |-- writingScore: integer (nullable = true)
 |-- finalScore: double (nullable = true)
 |-- admited: boolean (nullable = true)
 |-- parentEducationIndex: double (nullable = false)
 |-- preparationCourseIndex: double (nullable = false)

+--------------+------------+------------------+
|      features|readingScore|        prediction|
+--------------+------------+------------------+
|[0.0,32.0,0.0]|          38| 35.92040773033145|
|[0.0,37.0,0.0]|          39| 40.62788984312107|
|[0.0,42.0,0.0]|          49|  45.3353719559107|
|[0.0,43.0,0.0]|          43| 46.27686837846862|
|[0.0,46.0,0.0]|          57|49.101357646142404|
|[0.0,51.0,1.0]|          52|  51.6406952465792|
|[0.0,52.0,1.0

2.Vom folosi arbori de decizie pentru a prezice coloana admited. Coloana Admited reprezinta daca un student a fost admis sau nu. Arborele de decizie va fi antrenat pe baza celorlalte features(coloane): gender, race/ethnicity, parentEducation, lunch si preparationCourse pentru a prezice daca un student a fost admis sau nu. 


In [58]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

spark = SparkSession.builder.appName("DecisionTreeExample").getOrCreate()

data = spark.read.csv("/content/Students/StudentsPerformanceFormatted2/", header=True, inferSchema=True)

#Convertim variabilele categorice in reprezentari numerice folosind StringIndexer
indexers = [
    StringIndexer(inputCol=column, outputCol=column + "_index").fit(data)
    for column in ["gender", "race/ethnicity", "parentEducation", "lunch", "preparationCourse"]
]
indexed_data = data
for indexer in indexers:
    indexed_data = indexer.transform(indexed_data)

# Asamblam coloanele intr-un vector
feature_columns = ["gender_index", "race/ethnicity_index", "parentEducation_index", "lunch_index", "preparationCourse_index"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
indexed_data_assembled = assembler.transform(indexed_data)

# Convertim coloana admited din boolean in numeric
indexed_data_assembled = indexed_data_assembled.withColumn("admited", indexed_data_assembled["admited"].cast("integer"))

# Impartim datele in seturile de testare si de antrenare
train_data, test_data = indexed_data_assembled.randomSplit([0.8, 0.2], seed=42)

# Cream o instanta de decision tree classifier
decision_tree = DecisionTreeClassifier(labelCol="admited", featuresCol="features", seed=42)

#Antrenam modelul pe datele de train
model = decision_tree.fit(train_data)

#Facem predictii pe setul de test
predictions = model.transform(test_data)

# Evaluam modelul folosind BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="admited")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)
predictions.select("admited", "prediction", "probability").show()
#predictions.select("admited", "prediction", "probability").show(predictions.count(), False)


Accuracy: 0.6535211267605634
+-------+----------+--------------------+
|admited|prediction|         probability|
+-------+----------+--------------------+
|      1|       0.0|           [1.0,0.0]|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.33333333333333...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.33333333333333...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      0|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
|      1|       1.0|[0.18081180811808...|
|      1|       1.0|[0.18081180811808...|
|      0|       1.0|[0.07434944237918...|
|      1|       1.0|[0.07434944237918...|
+----